[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1T-sEv3HDSzldT0M-yEpl6DjRlyHlE1eq?usp=sharing)

# Group Project Milestone #1
### Mac Ambler
### Group Name: IMDb Analytics
### DATA 340-02, Spring 2023
### 04/13/2023

### SUBTASK: Data Extraction/Cleaning

My goal for this week was to load the data for the project into Python and write code that succesfully creates a nested dictionary of each director, their movies, and the crews involved in these projects. I'm happy to say that I have mostly accomplished this goal! The following is a step-by-step walkthrough of my progress to this point:

First, I imported all necessary packages for both the imdb_scraper code as well as my own, and read in the data from the GitHub link.



```
!pip install isoduration
!pip install NwalaTextUtils
!pip install PyMovieDb

!cp /content/drive/Shareddrives/IMDB_Analytics_Project/imdb_scraper.py .

import imdb_scraper
import json

import pandas as pd

url = 'https://raw.githubusercontent.com/anwala/teaching-network-science/main/spring-2023/homework/hw5-group/group-project-topic-01/100_film_directors.csv'
df = pd.read_csv(url)
```



Next, I created a new column in the dataframe with the isolated URI codes for the directors.



```
imdb_uri = []
for i in df['IMDb_URI']:
  imdb_uri.append(i[-10:][:-1])

df['URI'] = imdb_uri
```



The bulk of my efforts then were focused on creating this next section of code, which takes the directors' URIs and builds a nested dictionary with the following structure:

Director URI
--> Director Information
--> Movies
--> Movie Information/URIs
--> Crewmember Roles
--> Crewmember Names & URIs





```
dir_data = {}
start = 'title/'
start2 = 'name/'
end = '/?ref'


for i in df['URI']:
  dir_data[i] = {}
  dir_data.get(i)['Name'] = df[df['URI']==i]['FirstName'].values[0] + ' ' + df[df['URI']==i]['LastName'].values[0]
  dir_data.get(i)['Sex'] = df[df['URI']==i]['Sex'].values[0]
  dir_data.get(i)['Ethnicity_Race'] = df[df['URI']==i]['Ethnicity_Race'].values[0]
  dir_data.get(i)['Labels'] = df[df['URI']==i]['Labels'].values[0]
  dir_data.get(i)['Movies'] = {}
  if imdb_scraper.get_full_credits_for_director(i).get('credits') is not None:
    for mov_info in imdb_scraper.get_full_credits_for_director(i).get('credits'):
      dir_data.get(i).get('Movies')[mov_info['title']] = {}
      dir_data.get(i).get('Movies').get(mov_info['title'])['uri'] = mov_info['uri'][mov_info['uri'].find(start)+len(start):mov_info['uri'].rfind(end)]
      dir_data.get(i).get('Movies').get(mov_info['title'])['roles'] = {}
      if imdb_scraper.get_full_crew_for_movie(dir_data.get(i).get('Movies').get(mov_info['title'])['uri']).get('full_credits') is not None:
        for credit in imdb_scraper.get_full_crew_for_movie(dir_data.get(i).get('Movies').get(mov_info['title'])['uri']).get('full_credits'):
          if credit.get('role') != 'Cast (in credits order) verified as complete':
            dir_data.get(i).get('Movies').get(mov_info['title'])['roles'][credit.get('role')] = {}
            if credit.get('crew') is not None:
              for member in credit.get('crew'):
                dir_data.get(i).get('Movies').get(mov_info['title'])['roles'][credit.get('role')][member.get('link')[member.get('link').find(start2)+len(start):member.get('link').rfind(end)]] = {}
                dir_data.get(i).get('Movies').get(mov_info['title'])['roles'][credit.get('role')][member.get('link')[member.get('link').find(start2)+len(start):member.get('link').rfind(end)]]['name'] = member.get('name')
```



Finally, I wrote this nested dictionary to a JSON file and distributed it to my fellow groupmates so, although there are still problems I need to address, they could begin to work with some data in pursuit of their subtask goals.



```
with open('dir_data.json', 'w') as fp:
    json.dump(dir_data, fp)
```



There were a couple of significant issues I ran into this week that I aim to solve in the coming weeks. Most notably, I was unable to utilize the provided is_feature_film() function as it was producing the following error:


```
ERROR:NwalaTextUtils.textutils:imdb_scraper.py, 163, (<class 'isoduration.parser.exceptions.EmptyDuration'>, EmptyDuration('No duration information provided'), <traceback object at 0x7f1e660cf080>)
```



As a result, the nested dictionary I created ended up being significantly large than necessary and it took my code about an hour and a half to extract it all.

Additionally, I am yet to properly filter out the cast from all of the crewmember role lists. This is something I expect will be a simple fix in the coming days/week. 

### **SUMMARY**

COMPLETED:

*   Imported packages
*   Read in data
*   Isolated director URIs
*   Created nested dictionary
*   Wrote dictionary to JSON file

GOALS MOVING FORWARD:

*   Solve is_feature_film() issue
*   Filter out all cast members
*   Provide complete JSON to team members









